In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import jupyros
import rospy
import rospkg
from rospy_tutorials.srv import AddTwoInts, AddTwoIntsResponse
from turtlesim.srv import Spawn
from random import randint
from math import radians
from ipywidgets import Image
from ipycanvas import hold_canvas, Canvas

# Add Two Ints

In [ ]:
rospy.init_node("adder")

In [ ]:
def handle_add_two_ints(req):
    print("Returning [%s + %s = %s]"%(req.a, req.b, (req.a + req.b)))
    return AddTwoIntsResponse(req.a + req.b)

In [ ]:
%%thread_cell

srv = rospy.Service('add_two_ints', AddTwoInts, handle_add_two_ints)

In [ ]:
jupyros.client('add_two_ints', AddTwoInts)

# Turtlesim

In [ ]:
# !rosrun turtlesim turtlesim_node

In [ ]:
jupyros.client('spawn', Spawn)

In [ ]:
def get_random_turtle():
    turtles_svg = ['hydro', 'indigo', 'kinetic', 'lunar']
    turtles_png = ['box-turtle', 'diamondback', 'electric', 'fuerte', 'groovy', 'hydro', 'indigo', 'jade',
                   'kinetic', 'lunar', 'melodic', 'noetic', 'robot-turtle', 'sea-turtle', 'turtle']
    
    # Find path to a random turtle image
    r = rospkg.RosPack()
    turtlesim_path = r.get_path('turtlesim')
    turtle_path = turtlesim_path + '/images/' + turtles_png[randint(0, len(turtles_png)-1)] + '.png'

    turtle_img = Image.from_file(turtle_path)

    return turtle_img


def spawn_turtle():
    spawn_canvas = Canvas(width=1600, height=1200, layout=dict(width="100%"))
    
    # Water
    spawn_canvas.fill_style = "#4556FF"
    spawn_canvas.fill_rect(0, 0, spawn_canvas.width, spawn_canvas.height)
    
    # Turtle
    turtle_img = get_random_turtle()   
    turtle_size = 100  
    turtle_canvas = Canvas(width=turtle_size, height=turtle_size)   
    turtle_canvas.draw_image(turtle_img, width=turtle_size)
    
    # Center of turtle and canvas from top left
    x = (spawn_canvas.width - turtle_size) / 2
    y = (spawn_canvas.height - turtle_size) / 2
    
    # Water canvas
    spawn_canvas.draw_image(turtle_img, x, y, width=turtle_size)

    return turtle_canvas, spawn_canvas

In [ ]:
display(turtle)

In [ ]:
import rostopic

pubs, _ = rostopic.get_topic_list()
pubs

# Retrieve name of last turtle spawned
last_turtle = pubs[-1][0].split('/')[1]
print(last_turtle)

### Spawn turtle

In [ ]:
turtle_canvas, water_canvas = spawn_turtle()
display(water_canvas)

### Translate and rotate turtle

In [ ]:
from time import sleep

# SUBSCRIBE TO CURRENT POSE

def move_turtle(turtle_canvas, water_canvas, x, y, theta, turtle_size=100): 
    with hold_canvas(water_canvas):
        
        # Clear old animation step
        water_canvas.clear()

        # New animation drawings
        water_canvas.fill_style = "#4556FF"
        water_canvas.fill_rect(0, 0, water_canvas.width, water_canvas.height)
        
        # Transform canvas
        water_canvas.translate(x, y)
        water_canvas.rotate(theta)
        
        # Offsets for turtle center
        x_offset = -turtle_size / 2
        y_offset = -turtle_size / 2
        
        water_canvas.draw_image(turtle_canvas, x=x_offset, y=y_offset, width=turtle_size)
        
        # Revert transformation
        water_canvas.rotate(-theta)
        water_canvas.translate(-x, -y)

In [ ]:
x = water_canvas.width // 2
y = water_canvas.height // 2

for i in range(360):
    move_turtle(turtle_canvas, water_canvas, x+i, y+i, theta=radians(i))
    sleep(0.01)

In [ ]:
# Current position of turtle
x_pos = canvas.width // 2
y_pos = canvas.height // 2

# Number of steps in your animation
steps_number = 200

offscreen_canvas = Canvas(width=turtle_size, height=turtle_size)
offscreen_canvas.draw_image(turtle_img, width=turtle_size)

for i in range(steps_number):
    with hold_canvas(canvas):
        # Clear the old animation step
        canvas.clear()

        # Perfom all your drawings here
        canvas.fill_style = '#4556FF'
        canvas.fill_rect(0, 0, canvas.width, canvas.height)

        canvas.draw_image(offscreen_canvas, x_pos+i, y_pos-i, width=turtle_size)

    # Animation frequency ~50Hz = 1000./50. milliseconds
    sleep(0.02)

In [ ]:
from geometry_msgs.msg import Twist

topic_name = '/turtle1/cmd_vel'

jupyros.publish(topic_name, Twist)

# Moving turtle animation

In [ ]:
from time import sleep

# Number of steps in your animation
steps_number = 200

# Note how `hold_canvas` now wraps the entire for-loop
for i in range(steps_number):
    with hold_canvas(canvas):
        # Clear the old animation step
        canvas[1].clear()

        # Perfom all your drawings here
#         canvas.fill_rect(0, 0, canvas.width, canvas.height)
        canvas[1].draw_image(turtle_img, x_pos+i, y_pos-i, width=turtle_width)

        # Animation frequency ~50Hz = 1000./50. milliseconds
        sleep(.02)

In [ ]:
canvas = Canvas(width=1600, height=1200, layout=dict(width="100%"))
display(canvas)


# Water
canvas.fill_style = "#4556FF"
canvas.fill_rect(0, 0, canvas.width, canvas.height)

with hold_canvas(canvas):
    for _ in range(20):
        turtle_width = 90
        x_pos = randint(0, canvas.width-1)
        y_pos = randint(0, canvas.height-1)
        theta = radians(90)

        turtle_img = get_random_turtle()
        
        canvas.translate(x_pos, y_pos)
        canvas.rotate(-theta)

        canvas.draw_image(turtle_img, width=turtle_width)
        
        canvas.rotate(theta)
        canvas.translate(-x_pos, -y_pos)
